In [1]:
%load_ext autoreload
%autoreload 2
%cd C:\MAD4AG
%matplotlib inline

C:\MAD4AG


In [2]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from sklearn.cluster import DBSCAN
import geopandas
import folium
from folium.plugins import HeatMapWithTime
import warnings

warnings.filterwarnings('ignore')


C:\Users\aglar\AppData\Local\anaconda3\envs\pycharm\lib\site-packages\geopandas\_compat.py:115: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [3]:
file_name = f'./dbs/big_geodata_se/stops.parquet'

#Read each batchs file
df = pd.read_parquet(file_name)

# DBSCAN


In [4]:
def add_individual_clusters(data):
    # represent points consistently as (lat, lon)
    coords = data[['lat', 'lng']].values

    # define the number of kilometers in one radian
    kms_per_radian = 6371.0088

    # define epsilon as 0.1 kilometers, converted to radians for use by haversine
    epsilon = 0.1 / kms_per_radian

    db = DBSCAN(eps=epsilon, min_samples=1, algorithm='ball_tree', metric='haversine').fit(np.radians(coords))
    cluster_labels = db.labels_
    data['cluster'] = cluster_labels + 1
    return data


In [5]:
tqdm.pandas()
df = df.groupby('uid').progress_apply(add_individual_clusters)

  0%|          | 0/322919 [00:00<?, ?it/s]

## Intersect MAD data and DeSO data

In [6]:
gdf = geopandas.GeoDataFrame(df, geometry=geopandas.points_from_xy(df.lng, df.lat), crs="EPSG:4326")

In [7]:
DeSO = geopandas.read_file(f'C:/Synthetic_population_new/caglar/synthetic_sweden/input/deso_statistik_shp/Bef_Kon_region.shp')

print(DeSO.crs)

DeSO.to_crs(4326, inplace=True)
print(DeSO.crs)

PROJCS["SWEREF99 TM",GEOGCS["SWEREF99",DATUM["SWEREF99",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6619"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",15],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","3006"]]
epsg:4326


In [8]:
gdf = gdf.sjoin(DeSO[['Deso','geometry']], how="left")

gdf.drop("index_right", axis=1, inplace=True)
#out of sweden
print('Stops outside Sweden: ', len(gdf[gdf['Deso'].isna()]))

gdf = gdf.dropna(subset=['Deso'])

df = pd.DataFrame(gdf.drop(columns='geometry'))


Stops outside Sweden:  115534


## Calculate mean of clusters and snap to the closest building

In [9]:

df_cluster = df.groupby(['uid','cluster'])[['lat','lng']].mean().reset_index()


## Snap the cluster centre to the closest building.

In [10]:
building_point = geopandas.read_file(f'C:/Synthetic_population_new/caglar/synthetic_sweden/input/Geodatabase.gdb', layer= 'Buildings_SpatJ_SU_1km_DESO_poi_wgs84')

building_point = pd.DataFrame(building_point[['TARGET_FID','ANDAMAL_1',
       'Deso', 'KNKOD',  'POINT_X_sweref99',
       'POINT_Y_sweref99', 'POINT_X', 'POINT_Y']])
building_point = building_point[building_point.ANDAMAL_1 != 699]
building_point.rename(columns={'POINT_Y': 'lat', 'POINT_X': 'lng'}, inplace=True)

In [11]:
#HERE
# df_cluster.to_parquet(f'./dbs/intermediate/df_cluster.parquet')
# building_point.to_parquet(f'./dbs/intermediate/building_point.parquet')


In [12]:
# df_cluster = pd.read_parquet(f'./dbs/intermediate/df_cluster.parquet')
# building_point = pd.read_parquet(f'./dbs/intermediate/building_point.parquet')


In [13]:
from lib.nearest_point import nearest_neighbor

In [14]:
closest_build = nearest_neighbor(df_cluster, building_point, return_dist=True)


In [15]:
closest_build= closest_build[['TARGET_FID', 'ANDAMAL_1', 'Deso', 'KNKOD', 'POINT_X_sweref99','POINT_Y_sweref99', 'lat', 'lng', 'distance']]

closest_build.rename(columns={'lat':'building_lat', 'lng':'building_lng'}, inplace=True)

df_cluster = df_cluster.join(closest_build)


In [16]:
df_cluster= df_cluster[['uid', 'cluster', 'lat', 'lng', 'TARGET_FID',
       'ANDAMAL_1', 'Deso', 'KNKOD', 'building_lng', 'building_lat', 'distance']]

df_cluster.rename(columns={'lat':'cluster_lat', 'lng':'cluster_lng'}, inplace=True)



### If the distance between the closest building and the cluster centre is more than 200m, use the coordinates of the cluster.

In [17]:
#HERE

#df_cluster.cluster_lat = np.where(df_cluster['distance'] <= 0.2 , df_cluster.building_lat, df_cluster.cluster_lat)

#df_cluster.cluster_lng = np.where(df_cluster['distance'] <= 0.2 , df_cluster.building_lng, df_cluster.cluster_lng)

#df_cluster.ANDAMAL_1 = np.where(df_cluster['distance'] >= 0.2 , 0, df_cluster.ANDAMAL_1)

In [18]:
df_cluster.ANDAMAL_1.value_counts(normalize=True)

130    0.295030
133    0.219509
499    0.170975
399    0.064451
799    0.042485
299    0.035745
319    0.022765
304    0.019617
131    0.016667
132    0.013852
240    0.010403
135    0.007948
313    0.007550
318    0.007460
247    0.007448
199    0.007206
317    0.007021
253    0.005804
310    0.005444
599    0.004514
320    0.004205
307    0.002449
301    0.002446
311    0.002014
246    0.001858
242    0.001843
249    0.001801
315    0.001770
314    0.001099
303    0.001088
309    0.000957
321    0.000956
302    0.000906
324    0.000882
243    0.000848
306    0.000685
248    0.000438
251    0.000336
316    0.000313
252    0.000282
322    0.000270
308    0.000190
312    0.000155
305    0.000146
250    0.000127
245    0.000028
241    0.000011
244    0.000003
Name: ANDAMAL_1, dtype: float64

In [19]:

df = pd.merge(df, df_cluster[['uid', 'cluster', 'cluster_lat', 'cluster_lng', 'building_lat','building_lng', 'ANDAMAL_1', 'distance']], on=['uid', 'cluster'], how='left' )

In [20]:
#HERE
df.to_parquet(f'./dbs/intermediate/stops_1.parquet')
